In [1]:
### img -> tf record withot pickling

import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import os

from keras.utils import to_categorical
from sklearn import preprocessing

from PIL import Image
import itertools
from tqdm import tqdm

import numpy as np
import skimage.io as io
import tensorflow as tf

from tqdm import tqdm
from dltk.io.preprocessing import *
import scipy.ndimage


import pickle

root = 'D:\segmentiation/'
ct_set = os.path.join(root,'ct_train_test/ct_train/')
mr_set = os.path.join(root,'mr_train_test/mr_train/')
filenames = os.listdir(ct_set)



C:\Users\Nakyilkim\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def pickle_iter(path):
    f = open(path, 'rb')
    unpickler = pickle.Unpickler(f)
    try:
        for i in range(9999999999):
            yield unpickler.load()
    except:
        f.close()
        print('pickle generator created')

        
def pad3d(array):
    height = array.shape[0]
    depth = array.shape[2]
    
    if (height - depth) % 2 :
        pad_front = int((height + 1 - depth) / 2)
        pad_back = int((height - 1 - depth) / 2)
    else:
        pad_front = pad_back = int((height - depth) / 2)
    
    npad = ((0,0),(0,0),(pad_front,pad_back))
    array_padding = np.pad(array, npad, 'constant', constant_values=(0))
    array_padding[array_padding<0] = 0
    
    return array_padding


def image_preprocess(image, new_size, mask=False):
    assert np.sum(image.shape==image.shape[0])!=3    
    
    ratio = new_size / image.shape[0]
    
    image = scipy.ndimage.zoom(image, zoom=ratio, order=0)

    if mask:
        channel = 7 + 1 #background
        image = image.reshape(-1)
        image = label_encoder.fit_transform(image)
        
        
        print("annotation shape",image.shape)
        print("unique value", np.unique(image))
        
        
        image = to_categorical(image, class_num)
        print("processed mask shape",image.shape)

        
    else:
        channel = 1
    # reshape to raw shape
    image = image.reshape((new_size,)*3 + (channel,))
    print("Reshaped shape",image.shape)
    
    return image

def nii_loader(path,file_name):
    file_path =  os.path.join(path,file_name)
    fn = os.listdir(file_path)
    image = (nib.load(file_path + '/' + fn[0]))
    return image


def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def write_image_annotation_pairs_to_tfrecord_from_gen(generator, tfrecords_filename):
    """Writes given image/annotation pairs to the tfrecords file.
    The function reads each image/anno'tation pair given filenames
    of image and respective annotation and writes it to the tfrecord
    file.
    Parameters
    ----------
    filename_pairs : array of tuples (img_filepath, annotation_filepath)
        Array of tuples of image/annotation filenames
    tfrecords_filename : string
        Tfrecords filename to write the image/annotation pairs
    """
    options = tf.python_io.TFRecordOptions(tf.python_io.TFRecordCompressionType.GZIP)
    writer = tf.python_io.TFRecordWriter(tfrecords_filename,options=options)
    
    print("Start writing tfrecord")


    #for img_path, annotation_path in filename_pairs:



    for i, record in enumerate(generator):
        print(i)



        #img = np.array(Image.open(img_path))

        #annotation = np.array(Image.open(annotation_path))

        # Unomment this one when working with surgical data
        # annotation = annotation[:, :, 0]

        # The reason to store image sizes was demonstrated
        # in the previous example -- we have to know sizes
        # of images to later read raw serialized string,
        
        # convert to 1d array and convert to respective
        # shape that image used to have.
        #height = img.shape[0] 
    
        img = record[0]
        annotation = record[1]

        height = img.shape[0] 

        #width = img.shape[1]
        width = img.shape[1] 


        #add depth 
        depth = img.shape[2]
        #print(depth)
        #print(img[0].shape) #288,288,140 


        #img_raw = img.tostring()
        img_raw=img.tostring()

        ## 여기서 에러
        print("img shape: {}, img_raw shape: {}".format(img.shape,len(img_raw)))

        ## 여기서 에러
        annotation_raw = annotation.tostring()
        print("annotation_ shape: {}, annotation_raw shape: {}".format(annotation.shape,len(annotation_raw)))

        #print(annotation[1].shape)
        #print(annotation[2].shape)


        #print(annotation_raw)
        #annotation_raw = annotation.tostring()

        example = tf.train.Example(features=tf.train.Features(feature={
            'height': _int64_feature(height),
            'width': _int64_feature(width),
            'depth': _int64_feature(depth), 
            'image_raw': _bytes_feature(img_raw),
            'mask_raw': _bytes_feature(annotation_raw)}))

        writer.write(example.SerializeToString())

    writer.close()
    
def write_image_annotation_pairs_to_tfrecord_from_listitr(pickle_itr_list, tfrecords_filename):
    """Writes given image/annotation pairs to the tfrecords file.
    The function reads each image/anno'tation pair given filenames
    of image and respective annotation and writes it to the tfrecord
    file.
    Parameters
    ----------
    filename_pairs : array of tuples (img_filepath, annotation_filepath)
        Array of tuples of image/annotation filenames
    tfrecords_filename : string
        Tfrecords filename to write the image/annotation pairs
    """
    
    print("Start writing tfrecord")
    options = tf.python_io.TFRecordOptions(tf.python_io.TFRecordCompressionType.GZIP)
    writer = tf.python_io.TFRecordWriter(tfrecords_filename,options=options)

    #for img_path, annotation_path in filename_pairs:
    
    
    for pickle_itr in pickle_itr_list:

        for i,record in enumerate(pickle_itr):
            print(i)
            print(record.keys())

            img = record['image']
            annotation = record['label']

            #img = np.array(Image.open(img_path))

            #annotation = np.array(Image.open(annotation_path))

            # Unomment this one when working with surgical data
            # annotation = annotation[:, :, 0]

            # The reason to store image sizes was demonstrated
            # in the previous example -- we have to know sizes
            # of images to later read raw serialized string,
            # convert to 1d array and convert to respective
            # shape that image used to have.
            #height = img.shape[0] 

            print(img.shape)
            ##### img = img.reshape()
            
            
            height = img.shape[0] 

            #width = img.shape[1]
            width = img.shape[1] 


            #add depth 
            depth = img.shape[2]
            #print(depth)
            #print(img[0].shape) #288,288,140 


            #img_raw = img.tostring()
            img_raw=img.tostring()

            ## 여기서 에러
            print("img shape: {}, img_raw shape: {}".format(img.shape,len(img_raw)))

            ## 여기서 에러
            annotation_raw = annotation.tostring()
            print("annotation_ shape: {}, annotation_raw shape: {}".format(annotation.shape,len(annotation_raw)))

            #print(annotation[1].shape)
            #print(annotation[2].shape)


            #print(annotation_raw)
            #annotation_raw = annotation.tostring()

            example = tf.train.Example(features=tf.train.Features(feature={
                'height': _int64_feature(height),
                'width': _int64_feature(width),
                'depth': _int64_feature(depth), 
                'image_raw': _bytes_feature(img_raw),
                'mask_raw': _bytes_feature(annotation_raw)}))

            writer.write(example.SerializeToString())

    writer.close()
    
def read_image_annotation_pairs_from_tfrecord_itr_from_compressed(tfrecords_filename):
    """Return image/annotation pairs from the tfrecords file.
    The function reads the tfrecords file and returns image
    and respective annotation matrices pairs.
    Parameters
    ----------
    tfrecords_filename : string
        filename of .tfrecords file to read from
    
    Returns
    -------
    image_annotation_pairs : array of tuples (img, annotation)
        The image and annotation that were read from the file
    """
    image_annotation_pairs = []
    
    ## option...
    options = tf.python_io.TFRecordOptions(tf.python_io.TFRecordCompressionType.GZIP)

    record_iterator = tf.python_io.tf_record_iterator(path=tfrecords_filename,options=options)

    for string_record in record_iterator:

        example = tf.train.Example()
        example.ParseFromString(string_record)

        height = int(example.features.feature['height']
                                     .int64_list
                                     .value[0])

        width = int(example.features.feature['width']
                                    .int64_list
                                    .value[0])
        
        depth = int(example.features.feature['depth']
                                    .int64_list
                                    .value[0])

        img_string = (example.features.feature['image_raw']
                                      .bytes_list
                                      .value[0])
        

        annotation_string = (example.features.feature['mask_raw']
                                    .bytes_list
                                    .value[0])
        
        # img_1d = np.fromstring(img_string, dtype=np.uint8)
        img_1d = np.fromstring(img_string)
                               
        print("img_string shape",img_1d.shape)
        print("image :  {}, annotation length : {}".format(len(img_string),len(annotation_string)))

        #        img = img_1d.reshape((height, width, depth, -1))

        img = img_1d.reshape((height, width, depth,1))

        annotation_1d = np.fromstring(annotation_string, dtype=np.float32)
#        annotation_1d = np.fromstring(annotation_string)

        # Annotations don't have depth (3rd dimension)
        # TODO: check if it works for other datasets
        #  annotation = annotation_1d.reshape((height, width, depth,-1))

        annotation = annotation_1d.reshape((height, width, depth, 8))
        
        yield img, annotation

In [3]:
def pre_process_tfrecord_write_itr(path,itr,new_size):
    for record in tqdm(itr):
        file_name = record['name']
        img_fname = record['image']
        label_fname = record['label']
        
        img = nii_loader(path,img_fname)
        lab = nii_loader(path, label_fname)
        
        print("loaded Image Shape",img.get_data().shape,)
        
        img = pad3d(img.get_data())
        img = image_preprocess(img,new_size=new_size)
        
        lab = pad3d(lab.get_data())
        lab = image_preprocess(lab,new_size=new_size,mask=True)
        
        print("Mask shape when preproceesing ends",lab.shape)
        
        yield img, lab

        


## TF record Encoding

In [11]:
## Pickling 안하면 메모리 에러 ;;
## 성능 좋으면 여기만 돌리면 됨


ct_set = os.path.join(root,'ct_train_test/ct_train/')
ct_set_test = os.path.join(root,'ct_train_test/ct_test/')
mr_set = os.path.join(root,'mr_train_test/mr_train/')
mr_set_test =  os.path.join(root,'ct_train_test/mr_test/')

## 여기 바꾸면서 ct, mr 파일 만들기
temp_path = ct_set

name_list = []
filenames = os.listdir(temp_path)

for i in filenames:
    if i[:13] not in name_list:
        name_list.append(i[:13])
        
total_list = []
for i in name_list:
    temp_name = i
    temp_image = i+'_image.nii'
    temp_label = i+'_label.nii'
    total_list.append({'name' : temp_name,'image':temp_image,'label':temp_label})
    
print("starting gen")
preprocess_gen = pre_process_tfrecord_write_itr(temp_path,total_list,new_size=256)


out_file = temp_path.split('/')[-2] + '_tfrecord'
tfrecords_filename =os.path.join(root, out_file)

label_encoder = preprocessing.LabelEncoder()
class_num=7+1
print("write tfrecord")
write_image_annotation_pairs_to_tfrecord_from_gen(preprocess_gen,tfrecords_filename)

starting gen
write tfrecord
Start writing tfrecord


  0%|                                                                                           | 0/20 [00:00<?, ?it/s]

loaded Image Shape (512, 512, 363)
Reshaped shape (256, 256, 256, 1)
annotation shape (16777216,)
unique value [0 1 2 3 4 5 6 7]
processed mask shape (16777216, 8)
Reshaped shape (256, 256, 256, 8)
Mask shape when preproceesing ends (256, 256, 256, 8)
0
img shape: (256, 256, 256, 1), img_raw shape: 134217728
annotation_ shape: (256, 256, 256, 8), annotation_raw shape: 536870912


  5%|████▏                                                                              | 1/20 [00:35<11:15, 35.56s/it]

loaded Image Shape (512, 512, 239)
Reshaped shape (256, 256, 256, 1)
annotation shape (16777216,)
unique value [0 1 2 3 4 5 6 7]
processed mask shape (16777216, 8)
Reshaped shape (256, 256, 256, 8)
Mask shape when preproceesing ends (256, 256, 256, 8)
1
img shape: (256, 256, 256, 1), img_raw shape: 134217728
annotation_ shape: (256, 256, 256, 8), annotation_raw shape: 536870912


...

 90%|█████████████████████████████████████████████████████████████████████████▊        | 18/20 [10:10<01:08, 34.03s/it]

loaded Image Shape (512, 512, 297)
Reshaped shape (256, 256, 256, 1)
annotation shape (16777216,)
unique value [0 1 2 3 4 5 6 7]
processed mask shape (16777216, 8)
Reshaped shape (256, 256, 256, 8)
Mask shape when preproceesing ends (256, 256, 256, 8)
18
img shape: (256, 256, 256, 1), img_raw shape: 134217728
annotation_ shape: (256, 256, 256, 8), annotation_raw shape: 536870912


 95%|█████████████████████████████████████████████████████████████████████████████▉    | 19/20 [10:45<00:34, 34.27s/it]

loaded Image Shape (512, 512, 363)
Reshaped shape (256, 256, 256, 1)
annotation shape (16777216,)
unique value [0 1 2 3 4 5 6 7]
processed mask shape (16777216, 8)
Reshaped shape (256, 256, 256, 8)
Mask shape when preproceesing ends (256, 256, 256, 8)
19
img shape: (256, 256, 256, 1), img_raw shape: 134217728
annotation_ shape: (256, 256, 256, 8), annotation_raw shape: 536870912


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [11:27<00:00, 36.43s/it]


## Decoding test

In [12]:
tfr_itr = read_image_annotation_pairs_from_tfrecord_itr_from_compressed(tfrecords_filename)

In [13]:
for i,j in tfr_itr:
    decoded_image = i
    decoded_annotation = j
    break

preprocess_gen = pre_process_tfrecord_write_itr(temp_path,total_list,new_size=256)

for a,b in preprocess_gen:
    image = a
    annotation = b
    break

C:\Users\Nakyilkim\Anaconda3\lib\site-packages\ipykernel_launcher.py:292: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


img_string shape (16777216,)
image :  134217728, annotation length : 536870912


C:\Users\Nakyilkim\Anaconda3\lib\site-packages\ipykernel_launcher.py:301: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  0%|                                                                                           | 0/20 [00:00<?, ?it/s]

loaded Image Shape (512, 512, 363)
Reshaped shape (256, 256, 256, 1)
annotation shape (16777216,)
unique value [0 1 2 3 4 5 6 7]
processed mask shape (16777216, 8)
Reshaped shape (256, 256, 256, 8)
Mask shape when preproceesing ends (256, 256, 256, 8)


NameError: name 'decoded_annoatation' is not defined

In [14]:
decoded_image ==  image
decoded_annotation == annotation


array([[[[ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         ...,
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True]],

        [[ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         ...,
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True]],

        [[ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         ...,
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  T

## For Memory Error (Optional)

In [12]:
## For Low memory Setting
## preprocessing -> pickling -> tfrecord

outpath = r"D:\segmentiation\scipy_pickle_test0.pkl"
memory_error_list = []
cnt = 0

label_encoder = preprocessing.LabelEncoder()
class_num=7+1
## Making padded image

todolist = [i for i in total_list]



with open(outpath,"wb") as f:
    pickler = pickle.Pickler(f)
    for  record in tqdm(todolist):


        try :

            file_name = record['name']
            img_fname = record['image']
            label_fname = record['label']



            # nii load // 이 결과가가 원래 결과랑 같은 지 확인 하자
            img = nii_loader(ct_set,img_fname)
            lab = nii_loader(ct_set, label_fname)


            img = pad3d(img.get_data())
            img = image_preprocess(img,new_size=256)

            lab = pad3d(lab.get_data())
            lab = image_preprocess(lab,new_size=256,mask=True)



            print("Start MASKING"+"====="*10)

            List = {'filename':file_name,'image': img, 'label':  lab}
            pickler.dump(List)

            del file_name, img_fname, label_fname, img, lab
            
            print("Dumping",cnt)
            cnt += 1

        except MemoryError:
            print("Memory Error")
            todolist = total_list[cnt:]
            break

outpath = outpath[:-5] + str(int(outpath[-5]) + 1) + str(".pkl")



  0%|                                                                                           | 0/20 [00:00<?, ?it/s]

image shape (16777216,)
unique value [0 1 2 3 4 5 6 7]
Start MASKING==================================================




  5%|████▏                                                                              | 1/20 [00:35<11:17, 35.67s/it]

image shape (16777216,)
unique value [0 1 2 3 4 5 6 7]
Start MASKING==================================================




 10%|████████▎                                                                          | 2/20 [01:11<10:42, 35.71s/it]

...



 35%|█████████████████████████████                                                      | 7/20 [04:14<07:55, 36.57s/it]

image shape (16777216,)
unique value [0 1 2 3 4 5 6 7]
Start MASKING==================================================




 40%|█████████████████████████████████▏                                                 | 8/20 [04:48<07:09, 35.83s/it]

image shape (16777216,)
unique value [0 1 2 3 4 5 6 7]
Start MASKING==================================================




 45%|█████████████████████████████████████▎                                             | 9/20 [05:28<06:48, 37.14s/it]

In [24]:
with open(outpath,"wb") as f:
    pickler = pickle.Pickler(f)
    for  record in tqdm(todolist):


        try :

            file_name = record['name']
            img_fname = record['image']
            label_fname = record['label']



            # nii load // 이 결과가가 원래 결과랑 같은 지 확인 하자
            img = nii_loader(ct_set,img_fname)
            lab = nii_loader(ct_set, label_fname)


            img = pad3d(img.get_data())
            img = image_preprocess(img,new_size=256)

            lab = pad3d(lab.get_data())
            lab = image_preprocess(lab,new_size=256,mask=True)



            print("Start MASKING"+"====="*10)

            List = {'filename':file_name,'image': img, 'label':  lab}
            pickler.dump(List)

            del file_name, img_fname, label_fname, img, lab
            
            print("Dumping",cnt)
            cnt += 1

        except MemoryError:
            print("Memory Error")
            todolist = total_list[cnt:]
            break

outpath = outpath[:-4] + str(int(outpath[-5]) + 1) + str(".pkl")




  0%|                                                                                           | 0/11 [00:00<?, ?it/s]

image shape (16777216,)
unique value [0 1 2 3 4 5 6 7]
Start MASKING==================================================
Dumping 9





  9%|███████▌                                                                           | 1/11 [00:33<05:36, 33.69s/it]

image shape (16777216,)
unique value [0 1 2 3 4 5 6 7]
Start MASKING==================================================
Dumping 10





 18%|███████████████                                                                    | 2/11 [01:09<05:09, 34.40s/it]

...




 73%|████████████████████████████████████████████████████████████▎                      | 8/11 [05:03<01:57, 39.21s/it]

image shape (16777216,)
unique value [0 1 2 3 4 5 6 7]
Start MASKING==================================================
Dumping 17





 82%|███████████████████████████████████████████████████████████████████▉               | 9/11 [05:36<01:14, 37.45s/it]

Memory Error


ValueError: invalid literal for int() with base 10: '.'

In [67]:
with open(outpath,"wb") as f:
    pickler = pickle.Pickler(f)
    for  record in tqdm(todolist):

        try :

            file_name = record['name']
            img_fname = record['image']
            label_fname = record['label']



            # nii load // 이 결과가가 원래 결과랑 같은 지 확인 하자
            img = nii_loader(ct_set,img_fname)
            lab = nii_loader(ct_set, label_fname)


            img = pad3d(img.get_data())
            img = image_preprocess(img,new_size=256)

            lab = pad3d(lab.get_data())
            lab = image_preprocess(lab,new_size=256,mask=True)



            print("Start MASKING"+"====="*10)

            List = {'filename':file_name,'image': img, 'label':  lab}
            pickler.dump(List)

            del file_name, img_fname, label_fname, img, lab
            
            print("Dumping",cnt)
            cnt += 1

        except MemoryError:
            print("Memory Error")
            todolist = total_list[cnt:]
            break

outpath = outpath[:-4] + str(int(outpath[-4]) + 1) + str(".pkl")





  0%|                                                                                            | 0/2 [00:00<?, ?it/s]

image shape (16777216,)
unique value [0 1 2 3 4 5 6 7]
Start MASKING==================================================
Dumping 18






 50%|██████████████████████████████████████████                                          | 1/2 [00:33<00:33, 33.28s/it]

image shape (16777216,)
unique value [0 1 2 3 4 5 6 7]
Start MASKING==================================================
Dumping 19






100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [01:14<00:00, 35.60s/it]





In [78]:
pkl_itr1 = pickle_iter(r'D:\segmentiation\scipy_pickle_test0.pkl')
pkl_itr2 = pickle_iter(r'D:\segmentiation\scipy_pickle_test1.pkl')
pkl_itr3 = pickle_iter(r'D:\segmentiation\scipy_pickle_test2.pkl')

pkl_itr_list = [pkl_itr1, pkl_itr2, pkl_itr3]

tfrecords_filename = r"D:\segmentiation\scipy_tfrecord_compression.tfrecord"
write_image_annotation_pairs_to_tfrecord_from_listitr(pkl_itr_list, tfrecords_filename)


0
dict_keys(['filename', 'image', 'label'])
(256, 256, 256, 1)
img shape: (256, 256, 256, 1), img_raw shape: 134217728
annotation_ shape: (256, 256, 256, 8), annotation_raw shape: 536870912
1
dict_keys(['filename', 'image', 'label'])
(256, 256, 256, 1)
img shape: (256, 256, 256, 1), img_raw shape: 134217728
annotation_ shape: (256, 256, 256, 8), annotation_raw shape: 536870912
2
dict_keys(['filename', 'image', 'label'])
(256, 256, 256, 1)
img shape: (256, 256, 256, 1), img_raw shape: 134217728
annotation_ shape: (256, 256, 256, 8), annotation_raw shape: 536870912
3
dict_keys(['filename', 'image', 'label'])
(256, 256, 256, 1)
img shape: (256, 256, 256, 1), img_raw shape: 134217728
annotation_ shape: (256, 256, 256, 8), annotation_raw shape: 536870912
4
dict_keys(['filename', 'image', 'label'])
(256, 256, 256, 1)
img shape: (256, 256, 256, 1), img_raw shape: 134217728
annotation_ shape: (256, 256, 256, 8), annotation_raw shape: 536870912
5
dict_keys(['filename', 'image', 'label'])
(256, 